In [48]:
import pandas as pd
import numpy as np
import os
import datetime

In [4]:
PATH = '../../../../data/dota2/'

In [15]:
df_train_features = pd.read_csv(os.path.join(PATH, 'train_features.csv.zip'), index_col='match_id_hash')
df_test_features = pd.read_csv(os.path.join(PATH, 'test_features.csv.zip'), index_col='match_id_hash')
df_train_targets = pd.read_csv(os.path.join(PATH, 'train_targets.csv.zip'), index_col='match_id_hash')

df_train_features.shape, df_test_features.shape, df_train_targets.shape

((39675, 245), (10000, 245), (39675, 5))

In [12]:
df_train_features.head()

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
match_id_hash,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,22,7,1,11,11,0,0,0,0,...,0.000000,0,0,0,0,0.00,0,0,0,0
b9c57c450ce74a2af79c9ce96fac144d,658,4,0,3,10,15,7,2,0,7,...,0.000000,0,0,0,0,0.00,0,0,0,0
6db558535151ea18ca70a6892197db41,21,23,0,0,0,101,0,0,0,0,...,0.000000,0,0,0,0,0.00,0,0,0,0
46a0ddce8f7ed2a8d9bd5edcbb925682,576,22,7,1,4,14,1,0,3,1,...,8.664527,3,1,3,0,0.00,0,0,2,0
b1b35ff97723d9b7ade1c9c3cf48f770,453,22,7,1,3,42,0,1,1,0,...,0.000000,2,1,2,0,0.25,0,0,0,0


In [16]:
df_train_targets.head()

,game_time,radiant_win,duration,time_remaining,next_roshan_team
match_id_hash,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,False,992,837,NaN
b9c57c450ce74a2af79c9ce96fac144d,658,True,1154,496,NaN
6db558535151ea18ca70a6892197db41,21,True,1503,1482,Radiant
46a0ddce8f7ed2a8d9bd5edcbb925682,576,True,1952,1376,NaN
b1b35ff97723d9b7ade1c9c3cf48f770,453,False,2001,1548,NaN


In [52]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score

In [19]:
X = df_train_features.values
y = df_train_targets['radiant_win'].values

array([False,  True,  True, ..., False,  True,  True])

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)
len(X_train), len(X_test), y_train.size, y_test.size

(27772, 11903, 27772, 11903)

In [99]:
rf = RandomForestClassifier(n_estimators=2000, min_samples_leaf=2, random_state=17, n_jobs=-1)

In [100]:
%%time

rf.fit(X_train, y_train)

Wall time: 4min 5s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=2000,
                       n_jobs=-1, oob_score=False, random_state=17, verbose=0,
                       warm_start=False)

In [101]:
y_pred = rf.predict_proba(X_test)[:,1]
y_pred.size

11903

In [102]:
roc_auc_score(y_test, y_pred)

0.7920172544385998

In [103]:
accuracy_score(y_test, y_pred > 0.5)

0.7120053767957658

In [104]:
test_predict_submit(df_test_features, rf)

Submission saved to submission_2019-11-15_00-46-06.csv


In [60]:
def test_predict_submit(df_test_features, model):
    
    X_test = df_test_features.values
    y_test_pred = model.predict_proba(X_test)[:,1]
    df_submission = pd.DataFrame({'radiant_win_prob': y_test_pred}, index=df_test_features.index)
    
    submission_filename = 'submission_{}.csv'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
    df_submission.to_csv(submission_filename)
    
    print('Submission saved to {}'.format(submission_filename))

In [53]:
params = {'n_estimators':[100, 300, 500],
          'max_depth': range(1,5),
          'min_samples_leaf': range(1,5)
}
gscv = GridSearchCV(rf, param_grid=params, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True)

In [55]:
%%time

gscv.fit(X_train, y_train)

Wall time: 22min 17s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=1000, n_jobs=-1,
                                              oob_score=False, random_state=17,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'max_de

In [56]:
gscv.best_params_

{'max_depth': 4, 'min_samples_leaf': 2, 'n_estimators': 500}

In [57]:
gscv.best_score_

0.7636268722718619

In [59]:
gscv.best_estimator_

41

In [61]:
test_predict_submit(df_test_features, gscv.best_estimator_)

Submission saved to submission_2019-11-15_00-09-41.csv


In [106]:
!pip install xgboost

In [107]:
import xgboost as xgb

In [108]:
dtrain=xgb.DMatrix(df_train_features.values)
dftest=xgb.DMatrix(df_test_features.values)

In [113]:
%%time

param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)

XGBoostError: [01:20:11] C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:64: Check failed: info.labels_.Size() != 0U (0 vs. 0) : label set cannot be empty